<a href="https://colab.research.google.com/github/shashwatrajvanshi/Projects/blob/main/LLAVA_on_VIDEOS_to_identify_turns_of_a_vehicle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U transformers
!pip install bitsandbytes==0.41.3 accelerate==0.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 61.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (

In [ ]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from transformers import LlavaNextForConditionalGeneration, BitsAndBytesConfig
import torch
from PIL import Image
import requests
import cv2
import numpy as np
import math
import os
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
1
Tesla T4


In [ ]:
# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf", quantization_config=quantization_config,
    device_map="auto"

)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
import zipfile
import os

# Path to the zip file
zip_path = '/content/drive/MyDrive/leftvideo.zip'
extract_path = '/content/drive/MyDrive/leftvideos'

# Create a directory to extract the videos
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
import cv2
from PIL import Image
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming processor and model are already defined and loaded
# processor = ...
# model = ...

def get_video_frames(path, frame_rate=1):
    frames = []
    capture = cv2.VideoCapture(path)
    count = 0
    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(frame_rgb)
            frames.append((image, frame_rgb, count))
        count += 1
    capture.release()
    return frames

def save_frame(frame, frame_number, folder):
    """Saves a frame as an image file."""
    filename = os.path.join(folder, f"frame_{frame_number}.jpg")
    cv2.imwrite(filename, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

def generate_caption(pil_image, prompt):
    inputs = processor(prompt, pil_image, return_tensors="pt").to("cuda:0")
    output = model.generate(**inputs, max_new_tokens=1000)
    result = processor.decode(output[0], skip_special_tokens=True)
    caption = result.split("[/INST]", 1)[1]
    return caption.strip()

def classify_caption(caption):
    """Classify the caption as 'left turn' or 'straight'."""
    if "left turn" in caption.lower():
        return 1  # Left turn
    else:
        return 0  # Straight

def get_true_label(frame_count):
    # Implement this function to get the true label for the frame
    # For now, returning a dummy value
    return 0

def process_video(video_path, save_folder, frame_rate=1):
    print(f"Processing video: {video_path}")
    frames = get_video_frames(video_path, frame_rate)
    initial_prompt = "[INST] <image>\n Did the vehicle turn left?? [/INST]"
    results_new = []
    file_names = []
    true_labels = []  # Assuming you have true labels for evaluation

    for idx, (pil_image, rgb_frame, frame_count) in enumerate(frames):
        # Save the frame to the specified folder
        save_frame(rgb_frame, frame_count, save_folder)

        caption = generate_caption(pil_image, initial_prompt)
        print(caption)
        results_new.append(caption)
        file_names.append(f"{os.path.basename(video_path)}_frame_{frame_count}")

        # Classify the caption
        predicted_label = classify_caption(caption)
        true_label = get_true_label(frame_count)  # Implement this function to get the true label
        true_labels.append(true_label)

    return file_names, results_new, true_labels

def summarize_captions(captions):
    summary_prompt = "[INST]\n" + " ".join(captions) + "\n is the vehicle turning left?? [/INST]"
    summary = generate_caption(None, summary_prompt)  # Generate summary
    return summary

def evaluate_model(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    return accuracy, precision, recall, f1



In [ ]:
def main():
    # Set up paths
    path = '/content/drive/MyDrive/leftvideos'
    save_folder = '/content/drive/MyDrive/framesIVideo1'
    os.makedirs(save_folder, exist_ok=True)

    # Read video frames
    df = pd.DataFrame()
    video_files = [f for f in os.listdir(path) if f.endswith(".mp4")]
    all_results = []
    all_file_names = []
    all_true_labels = []
    all_predicted_labels = []

    for video in video_files:
        video_path = os.path.join(path, video)
        file_names, results_new, true_labels = process_video(video_path, save_folder, frame_rate=10)  # Adjust frame_rate as needed
        all_file_names.extend(file_names)
        all_results.extend(results_new)
        all_true_labels.extend(true_labels)
        all_predicted_labels.extend([classify_caption(caption) for caption in results_new])

    # Summarize the captions
    summary = summarize_captions(all_results)
    print("Summary of all frame outputs:", summary)

    df["Image_Name"] = all_file_names
    df["Prompt_Output"] = all_results
    df.to_excel("frame_output_media2.xlsx", index=False)

    # Save the summary to a text file
    with open("summary_output.txt", "w") as summary_file:
        summary_file.write(summary)

    # Evaluate the model
    accuracy, precision, recall, f1 = evaluate_model(all_true_labels, all_predicted_labels)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return 0

if __name__ == "__main__":
    main()

Processing video: /content/drive/MyDrive/leftvideos/0a0c3694-54476ae3.mp4


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Yes, the vehicle in the image appears to have turned left onto the street. You can tell by the position of the car relative to the road markings and the direction of the traffic flow.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Yes, the vehicle in the image appears to have turned left onto the street. You can tell by the position of the vehicle relative to the road markings and the direction of the traffic flow.


...

The image shows a vehicle's dashboard view, and it appears that the vehicle has turned left onto a street. The dashboard is displaying the vehicle's speedometer, which is showing a speed of 0, indicating that the vehicle is stationary. The steering wheel is turned to the left, which is consistent with a left turn. The headlights are on, and the road ahead is illuminated, suggesting that the vehicle is in a well-lit area, possibly at night.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The image shows a vehicle's dashboard with the view from the driver's perspective. It appears that the vehicle has turned left onto a street, as indicated by the steering wheel position and the orientation of the road ahead. The vehicle is now traveling on the left side of the road, which is consistent with a left turn.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The image shows a vehicle's dashboard with the hood of the car visible in the foreground. The dashboard indicates that the vehicle is in motion, as suggested by the speedometer and the illuminated headlights. The vehicle appears to be driving on a street with other cars parked on the side. There is no clear indication in the image that the vehicle has turned left. The dashboard does not show any turn signals or other indicators of a left turn.
